In [1]:
spark.conf.set("spark.sql.session.timeZone", 'Asia/Shanghai')
spark = SparkSession.builder.appName('demo-app').getOrCreate()

In [2]:
df = spark.read.option('header', 'True').csv('data/sales_train.csv')
df.count()
df.show(5)

+----------+--------------+-------+-------+----------+------------+
|      date|date_block_num|shop_id|item_id|item_price|item_cnt_day|
+----------+--------------+-------+-------+----------+------------+
|02.01.2013|             0|     59|  22154|     999.0|         1.0|
|03.01.2013|             0|     25|   2552|     899.0|         1.0|
|05.01.2013|             0|     25|   2552|     899.0|        -1.0|
|06.01.2013|             0|     25|   2554|   1709.05|         1.0|
|15.01.2013|             0|     25|   2555|    1099.0|         1.0|
+----------+--------------+-------+-------+----------+------------+
only showing top 5 rows



In [4]:
%%time
parquet_path = '/Users/lizeyu/Documents/Spark/output/sales'

df.repartition(10).write.partitionBy('shop_id').\
        mode('overwrite').option('compression', 'snappy').parquet(parquet_path)

CPU times: user 3.84 ms, sys: 2.73 ms, total: 6.57 ms
Wall time: 14.7 s


In [5]:
%%time
df_p = spark.read.option('basePath', parquet_path)\
    .parquet(parquet_path + '/shop_id=*/*.parquet')

CPU times: user 2.13 ms, sys: 1.86 ms, total: 3.99 ms
Wall time: 2.48 s


In [6]:
df_p.show(5)

+----------+--------------+-------+----------+------------+-------+
|      date|date_block_num|item_id|item_price|item_cnt_day|shop_id|
+----------+--------------+-------+----------+------------+-------+
|25.02.2013|             1|   4831|     699.3|         1.0|     31|
|05.08.2013|             7|  19799|     349.0|         1.0|     31|
|19.05.2013|             4|   3279|     299.0|         1.0|     31|
|31.05.2013|             4|  19369|     549.0|         1.0|     31|
|29.04.2013|             3|  19209|     149.0|         1.0|     31|
+----------+--------------+-------+----------+------------+-------+
only showing top 5 rows

